# Bibliotecas e Funções

In [1]:
# Usado para ler arquivos, carregar código personalizado e ter acesso a outros recursos de sistema
import os
import sys

# Manipulação e análise dos dados
import pandas as pd
import missingno as msno

# Recursos para visualização dos dados
import matplotlib.pyplot as plt
from IPython.core.display import HTML, display

# Carregar código personalizado disponível em ../src
sys.path.append(os.path.abspath(os.path.pardir))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.utils.experiments import set_dataset_split

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

# Análise Exploratória

A exploração dos dados será feita a partir do conjunto de treinamento gerado no notebook '01_Estruturacao.ipynb'. 

In [2]:
frame = pd.read_csv(os.path.join(settings.DATA_PATH, 'interim', 'training.csv'))

In [3]:
print(f'Registros: {len(frame)}.')

frame.head(10)

Registros: 32746.


,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category,period
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração,2015-11
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos long drink canecas,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia,2018-04
2,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê,2017-02
3,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração,2017-05
4,4336889,3436479,dia dos pais,1,37,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,2018-07-11 10:41:33,49.970000,208.0,1,1,1093,NaN,Lembrancinhas,2018-07
5,7544556,7118324,arranjo de flores para mesa,1,9,Arranjo de Flores - Orquidias,mini arranjos,2016-04-22 13:34:16,23.670000,207.0,1,5,276,NaN,Decoração,2016-04
6,10869150,5203458,lembrancinha maternidade,5,18,Kit Aromarizador + sacola / Lembrancinha Maternidade,bb lembrancinhas maternidade baby lembranca maternidade bebe conforto lembrancinha maternidade,2017-10-05 00:26:02,12.710000,55.0,0,33,1178,109.0,Lembrancinhas,2017-10
7,13193769,2933585,chaveiro dia dos pais,1,35,chaveiro dia dos pais,dia pais,2018-07-04 12:47:49,11.420000,6.0,1,23,72,NaN,Lembrancinhas,2018-07
8,13424151,8530613,manta personalizada,1,20,Manta para bebê personalizada de Nuvem com nome,nascimento manta baby cha bebe vestido bebe,2018-04-03 16:10:51,107.100000,9.0,1,1,639,26.0,Bebê,2018-04
9,12595651,5371868,pais,1,28,Chinelo Dia dos Pais,sandalia dia pais,2018-06-25 23:23:14,17.390000,6.0,1,22,933,NaN,Lembrancinhas,2018-06


## Análise de Mudanças entre Treino e Validação

No primeiro notebook, fez-se uma análise superficial do conteúdo do conjunto de dados e a avaliação de elementos que poderiam inflar as métricas de avaliação com o vazamento de informações dos dados de treino nos dados de teste. Com os dados de teste isolados, é possível simular uma nova divisão temporal entre os dados para identificar pontos de atenção que devem ser observados na hora de criar o modelo. Assim, serão criados dois conjuntos: pseudo-treinamento e pseudo-teste.

O ponto de corte pode ser obtido pela análise feita no primeiro notebook. Considerando que há uma diferença de ~15% entre o período de 2018-05 e 2018-08 (este segundo, é o limite atual do dataset), o primeiro período será utilizado como critério de corte.

In [4]:
cut_off_period = '2018-05'
split_frame = set_dataset_split(frame, cut_off_period)

pseudo_training_frame = split_frame.loc[lambda f: f['group'] != 'test'].drop(columns=['group'])
pseudo_test_frame = split_frame.loc[lambda f: f['group'] == 'test'].drop(columns=['group'])

print('Conjuntos de dados:')
print(f' - Treinamento Completo: {len(frame)}')
print(f' - Pseudo-Treino: {len(pseudo_training_frame)} ({100 * len(pseudo_training_frame) / len(frame):.2f}%)')
print(f' - Pseudo-Teste: {len(pseudo_test_frame)} ({100 * len(pseudo_test_frame) / len(frame):.2f}%)')

del split_frame

Conjuntos de dados:
 - Treinamento Completo: 32746
 - Pseudo-Treino: 27144 (82.89%)
 - Pseudo-Teste: 5602 (17.11%)


In [5]:
def extract_tags(frame: pd.DataFrame, column_name: str) -> set:
    tags = (frame
            .assign(tags=lambda f: f['concatenated_tags'].str.split())
            ['tags']
            .explode()
            .tolist())

    return tags

comparisons = []

for column in frame.columns:

    if column == 'concatenated_tags':
        training_values = set(extract_tags(pseudo_training_frame, 'concatenated_tags'))
        test_values = set(extract_tags(pseudo_test_frame, 'concatenated_tags'))
    else:
        training_values = set(pseudo_training_frame[column].tolist())
        test_values = set(pseudo_test_frame[column].tolist())

    comparisons.append((column,
                        len(training_values),
                        len(test_values),                        
                        len(training_values & test_values),                        
                        len(training_values | test_values),
                        len(training_values - test_values),
                        len(test_values - training_values)
                       )
                      )

comparisons_frame = pd.DataFrame(comparisons, columns=['Column', 'A', 'B', 'A and B', 'A or B','A - B',  'B - A'])
del comparisons

compounded_columns = ['A and B', 'A or B','A - B',  'B - A']

display(HTML('<h3>Comparar valores comuns entre Treinamento e Teste</h3>'))
print('Legenda:\n - A = Treinamento\n - B = Teste\n\n')
(comparisons_frame
 .assign(**{f'A and B %': lambda f: (f['A and B'] / f['A'] * 100).apply(lambda v: f'{v:.2f}%'),
            f'A or B %': lambda f: (f['A or B'] / f['A'] * 100).apply(lambda v: f'{v:.2f}%'),
            f'A - B %': lambda f: (f['A - B'] / f['A'] * 100).apply(lambda v: f'{v:.2f}%'),
            f'B - A %': lambda f: (f['B - A'] / f['A'] * 100).apply(lambda v: f'{v:.2f}%')
           })
 .style.set_properties(**{'width': '75px'})
)

Legenda:
 - A = Treinamento
 - B = Teste




,Column,A,B,A and B,A or B,A - B,B - A,A and B %,A or B %,A - B %,B - A %
0,product_id,21064,4092,0,25156,21064,4092,0.00%,119.43%,100.00%,19.43%
1,seller_id,6592,1996,1190,7398,5402,806,18.05%,112.23%,81.95%,12.23%
2,query,5623,2496,2170,5949,3453,326,38.59%,105.80%,61.41%,5.80%
3,search_page,5,5,5,5,0,0,100.00%,100.00%,0.00%,0.00%
4,position,39,39,39,39,0,0,100.00%,100.00%,0.00%,0.00%
5,title,18126,3713,330,21509,17796,3383,1.82%,118.66%,98.18%,18.66%
6,concatenated_tags,7095,3198,2595,7698,4500,603,36.58%,108.50%,63.42%,8.50%
7,creation_date,21060,4090,0,25150,21060,4090,0.00%,119.42%,100.00%,19.42%
8,price,12100,3676,2202,13574,9898,1474,18.20%,112.18%,81.80%,12.18%
9,weight,1090,499,426,1163,664,73,39.08%,106.70%,60.92%,6.70%


Como é possível observar, a separação entre os dados de treinamento e de teste expõe preocupações que devem existir caso se construa um modelo e o aplique a dados de produção.

Algumas preocupações que deverão ser consideradas:
 - Todos os produtos serão novos, então o modelo deve precisar generalizar o aprendizado para ter uma boa eficácia;
 - Existe repetição de vendedores (18%);
 - Existe grande repetição de entre as consultas dos dois conjuntos de dados (38%);
 - Seja página de busca ou posição, ambos os conjuntos possuem o mesmo conjunto de valores -- é interessante saber se foi algo controlado na extração ou se o comportamento dos usuários levou a isso;
 - Títulos de produtos são bastante diversos e possuem pouca interseção (~2%);
 - Há bastante tags, que foram comparadas individualmente, compartilhadas entre os dois conjuntos (~36%);
 - Como esperado, não há datas de criação em comum;
 - A comparação entre os preços e pesos não é a ideal, mas é possível encontrar valores em comum (18% e 39%). Para preço, deve-se esperar uma mudança do valor ao longo do tempo. Para ambos os atributos, pode-se fazer um estudo no próprio conjunto de treinamento para identificar mudanças de compartamento em geral ou para o mesmo produto do mesmo vendedor;
 - Como é uma coluna binária, a coluna pronta entrega  possui ambos os valores compartilhados;
 - Há diversidade maior de quantidade mínimas nos dados de treinamento e há algumas poucas inéditas nos dados de teste;
 - Há grande quantidade de contagens em comum (48%), para visualizações e pedidos, entre os conjuntos. Como os dados de teste possuem produtos mais novos e há o janelamento de 30 meses, há uma contribuição para a repetição dos valores.
 - As categorias, que são apenas 6, estão presentes em ambos os conjuntos, como verificado anteriormente;
 - São 112 meses na base de treino e 3 em teste, totalmente disjuntos. Aqui, é possível notar como o período de 3 meses do teste concentra uma quantidade grande de novos produtos em comparação com outros períodos do conjunto de treinamento. 
  
Dentre os itens observados, os que levantaram um ponto de atenção foram os itens query, título e tags. Como esses elementos, de natureza textual, devem ser importantes para ajudar a determinar a categoria do item ou a inteção do usuário, a baixa interseção de conteúdo entre os conjuntos de treinamento e de teste deve exigir cuidados adicionais para se lidar com palavras ou tags novas. Para isso, pode ser interessante trabalhar em um nível de sílabas ou transferir conhecimento de um conjunto de palavras mais amplo, seja de algo como um conjunto de dados externo, uma ontologia/tesauro, ou de um modelo de *embeddings* treinado em um conjunto de dados volumoso.
